The first step should be importing the needed packages and defining the function f(x) and our initial parameters 

In [78]:
import numpy as np
import pandas as pd
from numpy import random
import scipy

def func(inp):
    return 0.5*np.exp(-np.abs(inp))


    


In [79]:
def metropolis_hasting(x_0, N=10000, s=5):
    x_values = [x_0]
    for i in range(1,N):
        xi_minus1 = x_values[-1]
        x_star = random.normal(scale=s, loc=xi_minus1)
        r = func(x_star)/func(xi_minus1)
        u = random.uniform(0,1)

        if u < r:
            x_values.append(x_star)
        else:
            x_values.append(xi_minus1)
        
    return x_values            

         

In [80]:
x_values = metropolis_hasting(x_0=0)   
x_values[1:10]

[0,
 0,
 0,
 0.8212440639768005,
 0.8212440639768005,
 0.8212440639768005,
 0.8212440639768005,
 0.8212440639768005,
 0.8212440639768005]

Now for the second part, we introduce chains.

In [81]:
def mean(N=10000):
    return 1/N*sum(x_values)

print("mean:", mean())

def variance(N= 10000):
    mu = mean()  
    squared_diff = [(x - mu) ** 2 for x in x_values]  # Squared differences
    return 1/N*sum(squared_diff)

print("Variance:", variance())

mean: -0.019518168770717194
Variance: 1.8403029538465245


In [113]:
def multi_metropolis_hasting(J, x_0=0, N=2000, s=5):
    multi = []
    for i in range(J):
        chain_result = metropolis_hasting(x_0 + i, N, s)
        multi.append(chain_result)
    return multi

chains = multi_metropolis_hasting(J=5)

for i in range(1,5):
    print(chains[i][1:10])


[1, 1, 1, 1, 1, 0.9389802198608012, 0.9389802198608012, -0.3798589076615634, 2.0586863853285338]
[2, 2, 2, 2, 2, -3.432583317870386, -3.432583317870386, -3.432583317870386, 0.9917152306067294]
[3, 3.661609323266347, -2.2934242310786037, -2.356450402618066, -2.356450402618066, -2.356450402618066, -2.356450402618066, 2.398648000338367, 2.398648000338367]
[3.6804646483491026, 3.6804646483491026, 1.0593861186162692, 1.0593861186162692, 1.0593861186162692, 1.0593861186162692, 0.6495208559308769, 0.6983243573308595, 0.6983243573308595]


In [114]:
def overall_mean(J,chains, N=10000):
    means = []
    for i in range(J):
        individual_mean = 1/N*sum(chains[i])
        means.append(individual_mean)
        print(f"mean {i+1}: {individual_mean}")
    print(f"total mean: {sum(means)/J}\n")

overall_mean(J=5, chains=chains)


def overall_variance(J, chains, N=10000):
    variances = []
    

    for i in range(J):
        chain_mean = 1 / N * sum(chains[i])  
        squared_diff = [(x - chain_mean) ** 2 for x in chains[i]]  
        chain_variance = sum(squared_diff) / N  
        variances.append(chain_variance)
        print(f"Variance {i+1}: {chain_variance}")
    
    total_variance = sum(variances) / J
    print(f"Overall variance: {total_variance}")


overall_variance(J=4, chains=chains, N=2000)  
   

mean 1: 0.0031272281060742628
mean 2: 0.00791626999570815
mean 3: -0.0038980971109856734
mean 4: 0.011817627994997025
mean 5: 0.014294596826381397
total mean: 0.006651525162435032

Variance 1: 2.361598527145384
Variance 2: 1.584685954400375
Variance 3: 2.151291469784044
Variance 4: 1.8796217914978604
Overall variance: 1.994299435706916
